In [29]:
import pandas as pd  # Import pandas for data manipulation and analysis
import re  # Import the re module for regular expressions (used for text cleaning and pattern matching)
import os
from ast import literal_eval
from sklearn.preprocessing import StandardScaler

In [30]:
folder_path = "Data"
csv_files = ["avito_pfm1.csv", "avito_pfm2.csv"]

# Temporary list to store DataFrames
dataframes = []

# Read each file and append it to the list
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, encoding='utf-8')
    dataframes.append(df)

# Merge all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Display the first rows of the final DataFrame
merged_df.head()

,Prix,Année,BoiteàV,Carburant,kilometrage,Marque,Model,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements
0,155 000 DH,2023,Manuelle,Diesel,40 000 - 44 999,Renault,Express,5,WW au Maroc,Non,6 CV,Bon,[]
1,180 000 DH,2022,Automatique,Diesel,100 000 - 109 999,Dacia,Duster,5,WW au Maroc,Non,6 CV,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C..."
2,225 000 DH,2019,Automatique,Diesel,120 000 - 129 999,Hyundai,Tucson,5,WW au Maroc,Non,6 CV,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d..."
3,130 000 DH,2013,Manuelle,Diesel,200 000 - 249 999,Renault,Clio,5,WW au Maroc,Oui,6 CV,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]"
4,1 870 000 DH,2024,Automatique,Hybride,0 - 4 999,Porsche,Cayenne,Importée neuve,14 CV,Neuf,ABS,Airbags,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""S..."


In [31]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22842 entries, 0 to 22841
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Prix               22786 non-null  object
 1   Année              22713 non-null  object
 2   BoiteàV            22713 non-null  object
 3   Carburant          22704 non-null  object
 4   kilometrage        22702 non-null  object
 5   Marque             22700 non-null  object
 6   Model              22700 non-null  object
 7   NBporte            22700 non-null  object
 8   Origine            22368 non-null  object
 9   Première main      21953 non-null  object
 10  Puissance fiscale  21506 non-null  object
 11  Etat               20512 non-null  object
 12  équipements        22842 non-null  object
dtypes: object(13)
memory usage: 2.3+ MB


In [32]:
df = merged_df.copy()

Cleaning

In [33]:
df['kilometrage'] = df['kilometrage'].apply(lambda x: str(x).split(' - ')[0])
df['kilometrage'] = df['kilometrage'].str.replace(" 000", "")
df['kilometrage'] = pd.to_numeric(df['kilometrage'], errors='coerce')
df.head()

,Prix,Année,BoiteàV,Carburant,kilometrage,Marque,Model,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements
0,155 000 DH,2023,Manuelle,Diesel,40.0,Renault,Express,5,WW au Maroc,Non,6 CV,Bon,[]
1,180 000 DH,2022,Automatique,Diesel,100.0,Dacia,Duster,5,WW au Maroc,Non,6 CV,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C..."
2,225 000 DH,2019,Automatique,Diesel,120.0,Hyundai,Tucson,5,WW au Maroc,Non,6 CV,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d..."
3,130 000 DH,2013,Manuelle,Diesel,200.0,Renault,Clio,5,WW au Maroc,Oui,6 CV,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]"
4,1 870 000 DH,2024,Automatique,Hybride,0.0,Porsche,Cayenne,Importée neuve,14 CV,Neuf,ABS,Airbags,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""S..."


In [34]:
df['Prix'] = df['Prix'].str.replace(' DH', '')
df['Prix'] = df['Prix'].str.replace('\D', '', regex=True)
df = df[df['Prix'] != '']
df['Prix'] = df['Prix'].astype(float) / 1000
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\pc\AppData\Local\Temp\ipykernel_2508\2792952756.py:2: SyntaxWarning: invalid escape sequence '\D'
  df['Prix'] = df['Prix'].str.replace('\D', '', regex=True)


,Prix,Année,BoiteàV,Carburant,kilometrage,Marque,Model,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements
0,155.0,2023,Manuelle,Diesel,40.0,Renault,Express,5,WW au Maroc,Non,6 CV,Bon,[]
1,180.0,2022,Automatique,Diesel,100.0,Dacia,Duster,5,WW au Maroc,Non,6 CV,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C..."
2,225.0,2019,Automatique,Diesel,120.0,Hyundai,Tucson,5,WW au Maroc,Non,6 CV,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d..."
3,130.0,2013,Manuelle,Diesel,200.0,Renault,Clio,5,WW au Maroc,Oui,6 CV,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]"
4,1870.0,2024,Automatique,Hybride,0.0,Porsche,Cayenne,Importée neuve,14 CV,Neuf,ABS,Airbags,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""S..."


In [35]:
df['Puissance fiscale'] = df['Puissance fiscale'].astype(str).str.replace('CV', '', regex=False).str.strip()
df = df[df['Puissance fiscale'].str.replace('.', '', 1).str.isnumeric()]
df['Puissance fiscale'] = df['Puissance fiscale'].astype(float)
df = df[(df['Puissance fiscale'] >= 4) & (df['Puissance fiscale'] <= 20)]
df.head()

,Prix,Année,BoiteàV,Carburant,kilometrage,Marque,Model,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements
0,155.0,2023,Manuelle,Diesel,40.0,Renault,Express,5,WW au Maroc,Non,6.0,Bon,[]
1,180.0,2022,Automatique,Diesel,100.0,Dacia,Duster,5,WW au Maroc,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C..."
2,225.0,2019,Automatique,Diesel,120.0,Hyundai,Tucson,5,WW au Maroc,Non,6.0,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d..."
3,130.0,2013,Manuelle,Diesel,200.0,Renault,Clio,5,WW au Maroc,Oui,6.0,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]"
5,148.0,2016,Manuelle,Diesel,150.0,Ford,Focus,5,Importée neuve,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""O..."


In [36]:
df['Année'] = pd.to_numeric(df['Année'], errors='coerce')
df['Age'] = 2025 - df['Année']
df = df.drop(['Année'], axis=1)
df.head()

,Prix,BoiteàV,Carburant,kilometrage,Marque,Model,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements,Age
0,155.0,Manuelle,Diesel,40.0,Renault,Express,5,WW au Maroc,Non,6.0,Bon,[],2.0
1,180.0,Automatique,Diesel,100.0,Dacia,Duster,5,WW au Maroc,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C...",3.0
2,225.0,Automatique,Diesel,120.0,Hyundai,Tucson,5,WW au Maroc,Non,6.0,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d...",6.0
3,130.0,Manuelle,Diesel,200.0,Renault,Clio,5,WW au Maroc,Oui,6.0,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]",12.0
5,148.0,Manuelle,Diesel,150.0,Ford,Focus,5,Importée neuve,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""O...",9.0


In [37]:
df = df.rename(columns={'Model': 'Modèle'})

In [38]:
df.isnull().sum()

Prix                  2
BoiteàV               0
Carburant             0
kilometrage          88
Marque                0
Modèle                0
NBporte               0
Origine               0
Première main         0
Puissance fiscale     0
Etat                 45
équipements           0
Age                  24
dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17969 entries, 0 to 22841
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Prix               17967 non-null  float64
 1   BoiteàV            17969 non-null  object 
 2   Carburant          17969 non-null  object 
 3   kilometrage        17881 non-null  float64
 4   Marque             17969 non-null  object 
 5   Modèle             17969 non-null  object 
 6   NBporte            17969 non-null  object 
 7   Origine            17969 non-null  object 
 8   Première main      17969 non-null  object 
 9   Puissance fiscale  17969 non-null  float64
 10  Etat               17924 non-null  object 
 11  équipements        17969 non-null  object 
 12  Age                17945 non-null  float64
dtypes: float64(4), object(9)
memory usage: 1.9+ MB


In [40]:
df = df.dropna(subset=['Prix', 'kilometrage', 'NBporte', 'Puissance fiscale'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17879 entries, 0 to 22841
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Prix               17879 non-null  float64
 1   BoiteàV            17879 non-null  object 
 2   Carburant          17879 non-null  object 
 3   kilometrage        17879 non-null  float64
 4   Marque             17879 non-null  object 
 5   Modèle             17879 non-null  object 
 6   NBporte            17879 non-null  object 
 7   Origine            17879 non-null  object 
 8   Première main      17879 non-null  object 
 9   Puissance fiscale  17879 non-null  float64
 10  Etat               17834 non-null  object 
 11  équipements        17879 non-null  object 
 12  Age                17857 non-null  float64
dtypes: float64(4), object(9)
memory usage: 1.9+ MB


In [41]:
df['Etat'] = df['Etat'].fillna('Bon')

df['Origine'] = df['Origine'].fillna('Non spécifié')

# #Remplacer les valeurs manquantes 'Age' par la médiane
df['Age'] = df['Age'].fillna(df['Age'].median())
df.head()

,Prix,BoiteàV,Carburant,kilometrage,Marque,Modèle,NBporte,Origine,Première main,Puissance fiscale,Etat,équipements,Age
0,155.0,Manuelle,Diesel,40.0,Renault,Express,5,WW au Maroc,Non,6.0,Bon,[],2.0
1,180.0,Automatique,Diesel,100.0,Dacia,Duster,5,WW au Maroc,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""C...",3.0
2,225.0,Automatique,Diesel,120.0,Hyundai,Tucson,5,WW au Maroc,Non,6.0,Très bon,"[""Jantes aluminium"", ""Airbags"", ""Cam\u00e9ra d...",6.0
3,130.0,Manuelle,Diesel,200.0,Renault,Clio,5,WW au Maroc,Oui,6.0,Excellent,"[""Climatisation"", ""Vitres \u00e9lectriques""]",12.0
5,148.0,Manuelle,Diesel,150.0,Ford,Focus,5,Importée neuve,Non,6.0,Excellent,"[""Limiteur de vitesse"", ""Jantes aluminium"", ""O...",9.0


In [42]:
df.isnull().sum()

Prix                 0
BoiteàV              0
Carburant            0
kilometrage          0
Marque               0
Modèle               0
NBporte              0
Origine              0
Première main        0
Puissance fiscale    0
Etat                 0
équipements          0
Age                  0
dtype: int64

In [43]:
# 1. Définir les valeurs valides AVANT l'encodage
valeurs_valides = ['Bon', 'Très bon', 'Excellent', 'Neuf']

# 2. Supprimer les lignes où 'Etat' n'est pas dans les valeurs valides
df = df[df['Etat'].isin(valeurs_valides)]

In [44]:
# # Save the data to a CSV file 
# df = pd.DataFrame(df)
# df.to_csv('avito_pfm_cleaning.csv', index=False)

Encoding

In [45]:
# Remplacer les valeurs booléennes et textuelles
df["Première main"] = df["Première main"].map({"Oui": 1, "Non": 0})
df["Origine"] = df["Origine"].map({"WW au Maroc": 1, "Importée neuve": 2, "Dédouanée": 3})
df["Etat"] = df["Etat"].map({"Excellent": 3, "Très bon": 2, "Bon": 1})

# Normaliser les types de colonnes numériques
df['kilometrage'] = df['kilometrage'].astype(float)
df['Puissance fiscale'] = df['Puissance fiscale'].astype(float)
df['Age'] = df['Age'].astype(float)

# Convertir les équipements (colonne liste de strings)
df['équipements'] = df['équipements'].fillna("[]").apply(literal_eval)
for equip in df['équipements'].explode().dropna().unique():
    df[equip] = df['équipements'].apply(lambda x: equip in x)
df.drop(columns=['équipements'], inplace=True)
df['Etat'] = df['Etat'].replace({'Bon': 1, 'Très bon': 2, 'Excellent': 3, 'Neuf': 4})

In [46]:
# Encodage par fréquence
df["Marque_freq"] = df["Marque"].map(df["Marque"].value_counts() / len(df))
df["Modèle_freq"] = df["Modèle"].map(df["Modèle"].value_counts() / len(df))

df.drop(columns=["Marque", "Modèle"], inplace=True)

In [47]:
df.dropna(inplace=True)

In [48]:
df["BoiteàV"] = df["BoiteàV"].map({"Manuelle": 0, "Automatique": 1})

In [49]:
df["Carburant"] = df["Carburant"].map({
    "Diesel": 0,
    "Essence": 1,
    "Hybride": 2,
    "Electrique": 3,
    "LPG": 4
})

In [50]:
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [51]:
# Créer le scaler
scaler = StandardScaler()

# Normaliser les colonnes numériques continues
df[['kilometrage', 'Age', 'Puissance fiscale']] = scaler.fit_transform(df[['kilometrage', 'Age', 'Puissance fiscale']])

# Vérifier les changements
df.head()

,Prix,BoiteàV,Carburant,kilometrage,NBporte,Origine,Première main,Puissance fiscale,Etat,Age,...,Système de navigation/GPS,Radar de recul,ESP,ABS,Ordinateur de bord,Régulateur de vitesse,Sièges cuir,Toit ouvrant,Marque_freq,Modèle_freq
0,155.0,0,0,-1.091239,5,1.0,0,-0.743596,1.0,-1.110065,...,0,0,0,0,0,0,0,0,0.109372,0.007227
1,180.0,1,0,-0.347864,5,1.0,0,-0.743596,3.0,-0.973507,...,0,0,0,0,0,0,0,0,0.086553,0.021852
2,225.0,1,0,-0.100072,5,1.0,0,-0.743596,2.0,-0.563833,...,1,1,1,1,0,0,0,0,0.057076,0.014852
3,130.0,0,0,0.891095,5,1.0,1,-0.743596,3.0,0.255515,...,0,0,0,0,0,0,0,0,0.109372,0.038923
5,148.0,0,0,0.271616,5,2.0,0,-0.743596,3.0,-0.154159,...,1,1,1,1,1,1,0,0,0.045524,0.011267


In [52]:
print(df.dtypes)
df.head()

Prix                                  float64
BoiteàV                                 int64
Carburant                               int64
kilometrage                           float64
NBporte                                object
Origine                               float64
Première main                           int64
Puissance fiscale                     float64
Etat                                  float64
Age                                   float64
Limiteur de vitesse                     int32
Jantes aluminium                        int32
CD/MP3/Bluetooth                        int32
Climatisation                           int32
Vitres électriques                      int32
Verrouillage centralisé à distance      int32
Airbags                                 int32
Caméra de recul                         int32
Système de navigation/GPS               int32
Radar de recul                          int32
ESP                                     int32
ABS                               

,Prix,BoiteàV,Carburant,kilometrage,NBporte,Origine,Première main,Puissance fiscale,Etat,Age,...,Système de navigation/GPS,Radar de recul,ESP,ABS,Ordinateur de bord,Régulateur de vitesse,Sièges cuir,Toit ouvrant,Marque_freq,Modèle_freq
0,155.0,0,0,-1.091239,5,1.0,0,-0.743596,1.0,-1.110065,...,0,0,0,0,0,0,0,0,0.109372,0.007227
1,180.0,1,0,-0.347864,5,1.0,0,-0.743596,3.0,-0.973507,...,0,0,0,0,0,0,0,0,0.086553,0.021852
2,225.0,1,0,-0.100072,5,1.0,0,-0.743596,2.0,-0.563833,...,1,1,1,1,0,0,0,0,0.057076,0.014852
3,130.0,0,0,0.891095,5,1.0,1,-0.743596,3.0,0.255515,...,0,0,0,0,0,0,0,0,0.109372,0.038923
5,148.0,0,0,0.271616,5,2.0,0,-0.743596,3.0,-0.154159,...,1,1,1,1,1,1,0,0,0.045524,0.011267


In [53]:
df.isnull().sum()

Prix                                  0
BoiteàV                               0
Carburant                             0
kilometrage                           0
NBporte                               0
Origine                               0
Première main                         0
Puissance fiscale                     0
Etat                                  0
Age                                   0
Limiteur de vitesse                   0
Jantes aluminium                      0
CD/MP3/Bluetooth                      0
Climatisation                         0
Vitres électriques                    0
Verrouillage centralisé à distance    0
Airbags                               0
Caméra de recul                       0
Système de navigation/GPS             0
Radar de recul                        0
ESP                                   0
ABS                                   0
Ordinateur de bord                    0
Régulateur de vitesse                 0
Sièges cuir                           0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16782 entries, 0 to 22841
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Prix                                16782 non-null  float64
 1   BoiteàV                             16782 non-null  int64  
 2   Carburant                           16782 non-null  int64  
 3   kilometrage                         16782 non-null  float64
 4   NBporte                             16782 non-null  object 
 5   Origine                             16782 non-null  float64
 6   Première main                       16782 non-null  int64  
 7   Puissance fiscale                   16782 non-null  float64
 8   Etat                                16782 non-null  float64
 9   Age                                 16782 non-null  float64
 10  Limiteur de vitesse                 16782 non-null  int32  
 11  Jantes aluminium                    16782 non-

In [55]:
# # Save the data to a CSV file 
# df = pd.DataFrame(df)
# df.to_csv('avito_pfm_transformed.csv', index=False)

In [57]:
print(df.columns.tolist())

['Prix', 'BoiteàV', 'Carburant', 'kilometrage', 'NBporte', 'Origine', 'Première main', 'Puissance fiscale', 'Etat', 'Age', 'Limiteur de vitesse', 'Jantes aluminium', 'CD/MP3/Bluetooth', 'Climatisation', 'Vitres électriques', 'Verrouillage centralisé à distance', 'Airbags', 'Caméra de recul', 'Système de navigation/GPS', 'Radar de recul', 'ESP', 'ABS', 'Ordinateur de bord', 'Régulateur de vitesse', 'Sièges cuir', 'Toit ouvrant', 'Marque_freq', 'Modèle_freq']
